INDICACIONES INICIALES:

- Correr en GPU
- Las carpetas deben subirse en zip
- Recordar subir best. pt para entrenar
- El dataset utilizado para entrenar debe tener "cvat" en el nombre
- Los csv para cruzar el modelo con las dimensiones de los daños deben tener "paramento" y "losa" en los nombres

#
$$\textbf{Entrenamiento del modelo}$$

##
$$\textbf{Bloque 1 — Preparación del entorno:}
\\ \text{Crea la estructura de carpetas del proyecto en Colab (inputs, temporales, dataset, runs y resultados) e instala librerías.}$$


In [5]:
import os                                                         # Utilidades del sistema para rutas y directorios
import shutil                                                     # Operaciones para copiar, mover y borrar archivos
import zipfile                                                    # Manejo de archivos comprimidos .zip
import subprocess                                                 # Ejecución de comandos del sistema operativo
import glob                                                       # Búsqueda de archivos con patrones
import re                                                         # Manejo de expresiones regulares para texto
from pathlib import Path                                          # Manipulación de rutas de forma eficiente
import pandas as pd                                               # Procesamiento de datos en tablas (CSV)
import numpy as np                                                # Cálculos numéricos y procesos de interpolación
import cv2                                                        # Librería OpenCV para procesamiento de video
import torch                                                      # Framework para verificar disponibilidad de GPU
from google.colab import files                                    # Utilidad para subida y descarga de archivos

# Instalaciones de dependencias para el reporte y detección
!pip -q install ultralytics                                       # Instalación de YOLOv8
from ultralytics import YOLO                                      # Importación del motor de inferencia
!pip -q install fpdf                                              # Instalación de librería para PDFs
from fpdf import FPDF                                             # Importación del constructor de documentos

BASE = Path("/content")                                           # Ruta raíz de Colab
PATHS = {                                                         # Diccionario central de rutas
    "in": BASE/"in",                                              # Carpeta de entrada (Bloque 3)
    "work": BASE/"work",                                          # Carpeta para archivos temporales
    "out": BASE/"out",                                            # Carpeta para resultados finales
    "frames": BASE/"work/frames"                                  # Carpeta para capturas de fallas detectadas
}

for p in PATHS.values():                                          # Ciclo de creación de carpetas
    p.mkdir(parents=True, exist_ok=True)                          # Crea cada directorio si no existe

print(f"✅ Entorno preparado. GPU disponible: {torch.cuda.is_available()}") # Confirmación de estado técnico

✅ Entorno preparado. GPU disponible: False


##
$$\textbf{Bloque 4 — Exploración y control de espacio:} \
\\\ \text{Te muestra qué hay dentro de las carpetas del proyecto  y deja listas las carpetas de trabajo para frames y dataset crudo.}$$


In [ ]:
# Define una función simple para listar contenido de una carpeta sin tirar error
def listar_carpeta(ruta, max_items=50):                  # Crea una función para listar archivos/carpetas
    ruta = Path(ruta)                                    # Convierte la ruta a formato Path
    print("\n📁", ruta)                                   # Imprime la carpeta que se está revisando
    if not ruta.exists():                                # Si la carpeta no existe
        print("⚠️ No existe todavía.")                    # Avisa sin romper el notebook
        return                                            # Termina la función

    items = sorted(list(ruta.iterdir()), key=lambda p: (p.is_file(), p.name.lower()))  # Ordena primero carpetas
    if len(items) == 0:                                  # Si no hay nada dentro
        print("⚠️ Está vacía.")                           # Avisa sin romper el notebook
        return                                            # Termina la función

    for p in items[:max_items]:                          # Recorre hasta max_items elementos
        tipo = "DIR " if p.is_dir() else "FILE"          # Marca si es carpeta o archivo
        print(f" - {tipo} {p.name}")                     # Imprime el nombre

    if len(items) > max_items:                           # Si hay más de max_items
        print(f" ... y {len(items)-max_items} más")      # Avisa cuántos faltan por mostrar

# Lista las carpetas principales del proyecto para ver el estado general
print("✅ Listado rápido de carpetas del proyecto:")      # Mensaje de inicio
listar_carpeta("/content")                               # Muestra lo que hay en /content
listar_carpeta(DIR_IN)                                   # Muestra lo que hay en /content/in
listar_carpeta(DIR_WORK)                                 # Muestra lo que hay en /content/work
listar_carpeta(DIR_DATASET)                              # Muestra lo que hay en /content/dataset
listar_carpeta(DIR_RUNS)                                 # Muestra lo que hay en /content/runs
listar_carpeta(DIR_OUT)                                  # Muestra lo que hay en /content/out

# Muestra el espacio total del disco del entorno (si el comando está disponible)
print("\n✅ Espacio total del entorno (df -h):")           # Título del chequeo
try:
    subprocess.run(["df", "-h"], check=False)            # Ejecuta df sin romper si falla
except Exception:
    print("⚠️ No pude ejecutar df -h en este entorno.")   # Aviso si no se pudo

# Muestra cuánto pesa cada carpeta dentro de /content (para detectar qué está llenando)
print("\n✅ Peso por carpeta dentro de /content (du -h):") # Título del chequeo
try:
    subprocess.run(["bash", "-lc", "du -h --max-depth=1 /content | sort -h"], check=False)  # Ejecuta du ordenado
except Exception:
    print("⚠️ No pude ejecutar du en este entorno.")      # Aviso si no se pudo

# Detecta automáticamente qué tipo de archivo tienes en /content/in (sin tirar error si no hay nada)
in_files = list(DIR_IN.iterdir())                               # Lee todo lo que hay dentro de /content/in

# Busca un video por extensión común (si existe)
video = next((p for p in in_files if p.suffix.lower() in [".mp4", ".mov", ".avi", ".mkv"]), None)  # Encuentra el primer video

# Busca un zip que se llame images.zip (si existe)
zip_images = next((p for p in in_files if p.name.lower() == "images.zip"), None)  # Encuentra images.zip

# Busca un zip que parezca export de CVAT (si existe)
zip_cvat = next((p for p in in_files if p.suffix.lower() == ".zip" and "cvat" in p.name.lower()), None)  # Encuentra zip CVAT

# Define carpetas de trabajo estándar (frames y dataset_raw)
FRAMES_DIR = DIR_WORK/"frames"                                  # Carpeta donde se guardan frames extraídos del video
RAW_DIR = DIR_WORK/"dataset_raw"                                # Carpeta donde se descomprime el export de CVAT

# Crea las carpetas si no existen
FRAMES_DIR.mkdir(parents=True, exist_ok=True)                   # Crea /content/work/frames si no existe
RAW_DIR.mkdir(parents=True, exist_ok=True)                      # Crea /content/work/dataset_raw si no existe

# Imprime un resumen claro de lo que se detectó
print("✅ Detección de inputs en /content/in:")                  # Título del resumen
print(" - Video:", video.name if video else "No detectado")      # Informa si hay video
print(" - images.zip:", zip_images.name if zip_images else "No detectado")  # Informa si hay zip de imágenes
print(" - zip CVAT:", zip_cvat.name if zip_cvat else "No detectado")        # Informa si hay export CVAT

# Imprime rutas de trabajo listas
print("\n✅ Rutas de trabajo listas:")                           # Título para rutas
print(" - FRAMES_DIR:", FRAMES_DIR)                              # Ruta donde irán los frames
print(" - RAW_DIR:", RAW_DIR)                                    # Ruta donde irá el dataset crudo

# Mensaje guía para tu siguiente decisión (sin obligarte a nada)
print("\nℹ️ Próximo paso sugerido:")                              # Guía de flujo
if video:
    print(" - Tienes video: puedes extraer frames (Bloque 6).")   # Recomienda ruta A
elif zip_cvat:
    print(" - Tienes export CVAT: puedes descomprimir y normalizar dataset (Bloques 8+).")  # Recomienda ruta CVAT
elif zip_images:
    print(" - Tienes images.zip: puedes descomprimir para etiquetar (si lo necesitas).")    # Recomienda ruta B
else:
    print(" - Aún no hay inputs: puedes seguir armando notebook igual y subir después.")    # Recomienda seguir sin datos



✅ Listado rápido de carpetas del proyecto:

📁 /content
 - DIR  .config
 - DIR  dataset
 - DIR  in
 - DIR  out
 - DIR  runs
 - DIR  sample_data
 - DIR  work
 - FILE cvat dataset.zip

📁 /content/in
 - FILE best.pt
 - FILE cvat dataset (1).zip

📁 /content/work
⚠️ Está vacía.

📁 /content/dataset
⚠️ Está vacía.

📁 /content/runs
⚠️ Está vacía.

📁 /content/out
⚠️ Está vacía.

✅ Espacio total del entorno (df -h):

✅ Peso por carpeta dentro de /content (du -h):
✅ Detección de inputs en /content/in:
 - Video: No detectado
 - images.zip: No detectado
 - zip CVAT: cvat dataset (1).zip

✅ Rutas de trabajo listas:
 - FRAMES_DIR: /content/work/frames
 - RAW_DIR: /content/work/dataset_raw

ℹ️ Próximo paso sugerido:
 - Tienes export CVAT: puedes descomprimir y normalizar dataset (Bloques 8+).


##
$$\textbf{Bloque 8 — Importar export de CVAT:} \
\\\ \text{Si hay zip de CVAT (BLOQUE 3), lo descomprime en /content #/work/dataset_raw. Si no tiene cvat revisar bloque 6.}$$


In [ ]:
# Descomprime el export de CVAT (si existe) en /content/work/dataset_raw (si no existe, no falla: avisa)
# Nota: Este zip suele llamarse algo como "cvat_yolo_export.zip" (el nombre puede variar)

# Vuelve a buscar un zip que parezca de CVAT por si lo subiste recién
in_files = list(DIR_IN.iterdir())                                              # Lee archivos en /content/in
zip_cvat = next((p for p in in_files if p.suffix.lower() == ".zip" and "cvat" in p.name.lower()), None)  # Busca zip con "cvat" en el nombre

# Si no encontramos zip CVAT, avisamos y seguimos sin error
if zip_cvat is None:                                                           # Revisa si existe export de CVAT
    print("⚠️ No detecté ningún .zip de CVAT en /content/in (debe tener 'cvat' en el nombre).")           # Aviso
    print("ℹ️ Cuando lo tengas, súbelo y vuelve a correr este bloque.")         # Guía
else:
    # Limpia dataset_raw anterior para evitar mezclar versiones
    if RAW_DIR.exists():                                                       # Revisa si ya existe dataset_raw
        shutil.rmtree(RAW_DIR)                                                 # Borra dataset_raw anterior completo
    RAW_DIR.mkdir(parents=True, exist_ok=True)                                 # Crea dataset_raw limpio

    # Descomprime el zip de CVAT dentro de dataset_raw
    with zipfile.ZipFile(zip_cvat, "r") as z:                                  # Abre el zip de CVAT
        z.extractall(RAW_DIR)                                                  # Extrae todo su contenido en RAW_DIR

    # Lista contenido para confirmar que se extrajo algo
    extracted_any = any(RAW_DIR.rglob("*"))                                     # Verifica si hay archivos extraídos
    if not extracted_any:                                                      # Si no se extrajo nada
        print("⚠️ El zip se descomprimió, pero no veo archivos dentro. Revisa si el zip está correcto.")  # Aviso
    else:
        print("✅ Export CVAT descomprimido en:", RAW_DIR)                      # Confirmación
        # Muestra un vistazo rápido de archivos/carpetas extraídas
        top_items = sorted(list(RAW_DIR.iterdir()))                             # Lista el primer nivel de dataset_raw
        print("✅ Primer nivel dentro de dataset_raw:")                         # Título del listado
        for p in top_items[:30]:                                                # Muestra hasta 30 ítems
            tag = "DIR " if p.is_dir() else "FILE"                              # Marca si es carpeta o archivo
            print(" -", tag, p.name)                                            # Imprime nombre
        if len(top_items) > 30:                                                 # Si hay muchos ítems
            print(f" ... y {len(top_items)-30} más")                            # Indica que hay más


✅ Export CVAT descomprimido en: /content/work/dataset_raw
✅ Primer nivel dentro de dataset_raw:
 - DIR  cvat dataset


##
$$\textbf{Bloque 9 — Detectar (CVAT) + Normalizar YOLO + Split:} \
\\\ \text{Busca automáticamente imágenes y labels dentro de #dataset\_raw, y construye el split en #/content/dataset.}$$


In [ ]:
# Detecta imágenes/labels en dataset_raw y construye dataset YOLO final (train/val) creando .txt vacíos para negativos
import random                                                    # Sirve para mezclar antes del split

img_exts = {".jpg", ".jpeg", ".png"}                             # Extensiones válidas de imágenes
train_ratio = 0.8                                                # Proporción train/val
seed = 42                                                        # Semilla para split repetible

# Define rutas YOLO estándar de salida
IMG_TRAIN = DIR_DATASET/"images/train"                           # Imágenes train
IMG_VAL   = DIR_DATASET/"images/val"                             # Imágenes val
LBL_TRAIN = DIR_DATASET/"labels/train"                           # Labels train
LBL_VAL   = DIR_DATASET/"labels/val"                             # Labels val

# Crea carpetas de salida
for d in [IMG_TRAIN, IMG_VAL, LBL_TRAIN, LBL_VAL]:               # Recorre carpetas YOLO
    d.mkdir(parents=True, exist_ok=True)                         # Crea si falta

# Verifica que exista dataset_raw
if not RAW_DIR.exists():                                         # Si no existe dataset_raw
    print("⚠️ No existe /content/work/dataset_raw todavía. Corre el Bloque 8 (descomprimir CVAT) primero.")  # Aviso
else:
    # Busca imágenes y txt dentro de dataset_raw (recursivo)
    all_imgs = [p for p in RAW_DIR.rglob("*") if p.suffix.lower() in img_exts]   # Todas las imágenes
    all_txt  = [p for p in RAW_DIR.rglob("*.txt")]                               # Todos los txt

    print("✅ Archivos encontrados dentro de dataset_raw:")       # Resumen inicial
    print(" - Imágenes:", len(all_imgs))                         # Cantidad imágenes
    print(" - TXT:", len(all_txt))                               # Cantidad txt

    # Si falta algo, avisa pero no rompe
    if len(all_imgs) == 0:
        print("⚠️ No encontré imágenes dentro de dataset_raw. Revisa estructura del export (o tu zip).")  # Aviso
    if len(all_txt) == 0:
        print("⚠️ No encontré labels (.txt) dentro de dataset_raw. Ojo: CVAT solo exporta txt con anotaciones.")  # Aviso

    # Detecta carpeta con más imágenes y carpeta con más txt (candidatas principales)
    img_dir = None                                               # Carpeta candidata de imágenes
    lbl_dir = None                                               # Carpeta candidata de labels

    if len(all_imgs) > 0:                                        # Si hay imágenes
        counts_img_parent = {}                                   # Conteo por carpeta
        for p in all_imgs:                                       # Recorre imágenes
            counts_img_parent[p.parent] = counts_img_parent.get(p.parent, 0) + 1  # Cuenta
        img_dir = max(counts_img_parent, key=counts_img_parent.get)              # Elige mayor

    if len(all_txt) > 0:                                         # Si hay txt
        counts_lbl_parent = {}                                   # Conteo por carpeta
        for p in all_txt:                                        # Recorre txt
            counts_lbl_parent[p.parent] = counts_lbl_parent.get(p.parent, 0) + 1  # Cuenta
        lbl_dir = max(counts_lbl_parent, key=counts_lbl_parent.get)              # Elige mayor

    print("\n✅ Rutas detectadas (candidatas principales):")      # Imprime rutas
    print(" - img_dir:", str(img_dir) if img_dir else "No detectado")  # Carpeta imágenes
    print(" - lbl_dir:", str(lbl_dir) if lbl_dir else "No detectado")  # Carpeta labels

    # Si no se detectaron rutas, termina sin error
    if (img_dir is None) or (lbl_dir is None):
        print("⚠️ No pude detectar img_dir y/o lbl_dir. Revisa qué hay dentro de dataset_raw (Bloque 4 listar carpetas).")  # Aviso
    else:
        # Crea mapas por stem para hacer match imagen <-> label
        img_map = {}                                             # stem -> ruta imagen
        for p in img_dir.iterdir():                              # Recorre archivos en img_dir
            if p.suffix.lower() in img_exts:                     # Si es imagen
                img_map[p.stem] = p                              # Guarda

        lbl_map = {}                                             # stem -> ruta label
        for p in lbl_dir.iterdir():                              # Recorre archivos en lbl_dir
            if p.suffix.lower() == ".txt":                       # Si es txt
                lbl_map[p.stem] = p                              # Guarda

        # Diagnóstico de matching
        all_stems = sorted(img_map.keys())                       # Todas las imágenes (incluye negativos)
        paired = sorted(set(img_map.keys()) & set(lbl_map.keys()))            # Con label
        missing_lbl = sorted(set(img_map.keys()) - set(lbl_map.keys()))      # Sin label (negativos)
        missing_img = sorted(set(lbl_map.keys()) - set(img_map.keys()))      # Txt sin imagen

        print("\n✅ Matching imagen + label (incluyendo negativos):")  # Reporte matching
        print(" - Total imágenes:", len(all_stems))              # Total
        print(" - Con label (.txt):", len(paired))               # Con txt
        print(" - Sin label (se creará .txt vacío):", len(missing_lbl))  # Negativos
        print(" - Txt sin imagen (se ignoran):", len(missing_img))       # Huérfanos

        # Si no hay imágenes, no se puede construir dataset
        if len(all_stems) == 0:
            print("⚠️ No hay imágenes válidas para construir dataset. Revisa nombres/extensiones.")  # Aviso
        else:
            # Limpia salidas anteriores
            for d in [IMG_TRAIN, IMG_VAL, LBL_TRAIN, LBL_VAL]:   # Recorre carpetas de salida
                for f in d.glob("*"):                            # Recorre archivos dentro
                    f.unlink()                                   # Borra

            # Split train/val sobre TODAS las imágenes
            random.seed(seed)                                    # Semilla
            random.shuffle(all_stems)                             # Mezcla
            cut = int(len(all_stems) * train_ratio)              # Corte
            train_ids = all_stems[:cut]                          # Train stems
            val_ids = all_stems[cut:]                            # Val stems

            # Copia imagen y label (o crea vacío si falta)
            def copy_img_and_label(stem, img_dst, lbl_dst):      # Función copiar + label
                img_src = img_map[stem]                          # Imagen fuente
                shutil.copy2(img_src, img_dst / img_src.name)    # Copia imagen

                lbl_out = lbl_dst / f"{stem}.txt"                # Label destino
                if stem in lbl_map:                              # Si existe label real
                    shutil.copy2(lbl_map[stem], lbl_out)         # Copia label
                else:
                    lbl_out.write_text("", encoding="utf-8")     # Crea label vacío (negativo)

            # Copia a train
            for s in train_ids:                                  # Recorre train
                copy_img_and_label(s, IMG_TRAIN, LBL_TRAIN)      # Copia

            # Copia a val
            for s in val_ids:                                    # Recorre val
                copy_img_and_label(s, IMG_VAL, LBL_VAL)          # Copia

            # Resumen final
            n_train_img = len(list(IMG_TRAIN.glob("*")))         # Cuenta imgs train
            n_train_lbl = len(list(LBL_TRAIN.glob("*.txt")))     # Cuenta labels train
            n_val_img = len(list(IMG_VAL.glob("*")))             # Cuenta imgs val
            n_val_lbl = len(list(LBL_VAL.glob("*.txt")))         # Cuenta labels val

            print("\n✅ Dataset YOLO final creado en:", DIR_DATASET)  # Confirma creación
            print(" - Train: imgs =", n_train_img, "| lbls =", n_train_lbl)  # Resumen train
            print(" - Val:   imgs =", n_val_img,   "| lbls =", n_val_lbl)    # Resumen val

            # Chequeo 1 txt por imagen (ideal)
            if n_train_img != n_train_lbl:
                print("⚠️ Ojo: TRAIN imgs != txt (deberían ser iguales).")  # Aviso
            if n_val_img != n_val_lbl:
                print("⚠️ Ojo: VAL imgs != txt (deberían ser iguales).")    # Aviso

            # Guarda resumen para trazabilidad
            summary_path = DIR_OUT/"dataset_summary.txt"         # Archivo resumen
            summary_text = (
                f"total_imgs={len(all_stems)}\n"
                f"imgs_with_lbl={len(paired)}\n"
                f"imgs_without_lbl_created_empty={len(missing_lbl)}\n"
                f"txt_without_img_ignored={len(missing_img)}\n"
                f"train_ratio={train_ratio}\n"
                f"train_imgs={n_train_img}\ntrain_lbls={n_train_lbl}\n"
                f"val_imgs={n_val_img}\nval_lbls={n_val_lbl}\n"
            )
            summary_path.write_text(summary_text, encoding="utf-8")  # Escribe resumen
            print("✅ Resumen guardado en:", summary_path)        # Confirma guardado


✅ Archivos encontrados dentro de dataset_raw:
 - Imágenes: 885
 - TXT: 321

✅ Rutas detectadas (candidatas principales):
 - img_dir: /content/work/dataset_raw/cvat dataset/images
 - lbl_dir: /content/work/dataset_raw/cvat dataset/labels

✅ Matching imagen + label (incluyendo negativos):
 - Total imágenes: 885
 - Con label (.txt): 321
 - Sin label (se creará .txt vacío): 564
 - Txt sin imagen (se ignoran): 0

✅ Dataset YOLO final creado en: /content/dataset
 - Train: imgs = 708 | lbls = 708
 - Val:   imgs = 177 | lbls = 177
✅ Resumen guardado en: /content/out/dataset_summary.txt


##
$$\textbf{Bloque 11 — Crear data.yaml (receta del dataset):} \
\\\ \text{Genera el archivo data.yaml que le dice a YOLO dónde está tu dataset (train/val) y se crean manualmente las clases.}$$


In [ ]:
# Crea el archivo data.yaml para YOLO (si no hay dataset aún, no falla: avisa)
yaml_path = BASE/"data.yaml"                                    # Define la ruta donde se guardará el YAML

# Define aquí tus clases EXACTAS y en el MISMO orden que usaste en CVAT - OJO DEBE HACERSE MANUAL
classes = [
    "falla junta paramento izquierdo",
    "falla junta paramento derecho",
    "falla junta losa fondo",
    "estría lado izquierdo",
    "estría lado derecho",
    "estría centro",
    "daño paramento",
    "daño losa fondo",
]                                                               # Lista de nombres de clases (edítala tú)

# Verifica que existan carpetas train/val para evitar un YAML apuntando a nada
train_dir_ok = (DIR_DATASET/"images/train").exists()            # Revisa si existe la carpeta de imágenes train
val_dir_ok = (DIR_DATASET/"images/val").exists()                # Revisa si existe la carpeta de imágenes val

# Si no hay estructura dataset, avisa y no rompe el notebook
if not (train_dir_ok and val_dir_ok):                           # Si faltan carpetas básicas del dataset
    print("⚠️ No detecto la estructura de dataset en /content/dataset/images/train y /val.")  # Aviso
    print("ℹ️ Corre el Bloque 10 (normalización + split) antes de crear el data.yaml.")       # Guía
else:
    # Si no definiste clases, avisa para que no entrenes con un YAML incompleto
    if len(classes) == 0:                                       # Si la lista de clases está vacía
        print("⚠️ La lista 'classes' está vacía. Agrega tus clases en el orden de CVAT antes de entrenar.")  # Aviso
        print("ℹ️ Igual crearé el data.yaml, pero NO deberías entrenar hasta completar 'classes'.")          # Guía

    # Construye el contenido del YAML que YOLO necesita
    yaml_text = f"""path: {DIR_DATASET}
train: images/train
val: images/val
names:
"""                                                             # Texto base del YAML (path + rutas train/val)

    # Agrega las clases con su índice (0,1,2...) en el orden correcto
    for i, c in enumerate(classes):                              # Recorre clases con índice
        yaml_text += f"  {i}: {c}\n"                              # Agrega cada clase al YAML

    # Guarda el archivo data.yaml
    yaml_path.write_text(yaml_text, encoding="utf-8")            # Escribe el YAML en disco

    # Prints de confirmación + vista rápida del contenido
    print("✅ data.yaml creado en:", yaml_path)                   # Confirma ruta del archivo creado
    print("✅ Contenido de data.yaml:")                           # Título del contenido
    print(yaml_text)                                              # Muestra el texto completo


✅ data.yaml creado en: /content/data.yaml
✅ Contenido de data.yaml:
path: /content/dataset
train: images/train
val: images/val
names:
  0: falla junta paramento izquierdo
  1: falla junta paramento derecho
  2: falla junta losa fondo
  3: estría lado izquierdo
  4: estría lado derecho
  5: estría centro
  6: daño paramento
  7: daño losa fondo



##
$$\textbf{Bloque 12 — Entrenamiento YOLO:} \
\\\ \text{Entrena un modelo YOLO usando data.yaml y guarda los pesos (best.pt/last.pt) y métricas dentro de #/content/runs/train.}$$


In [ ]:
# Bloque 12 — Entrenamiento YOLO (Modo "Save Game" con Auto-Descarga)

from google.colab import files  # Necesario para la descarga automática

# 1. Configuración de parámetros
data_yaml = "/content/data.yaml"
batch_size = 16
epochs = 300
img_size = 640
output_dir = "/content/runs"
best_pt_path = Path("/content/in/best.pt") # Ruta de tu "Partida Guardada"

# Asegurar que la carpeta /content/in existe
best_pt_path.parent.mkdir(parents=True, exist_ok=True)

# 2. Lógica de carga de modelo (Continuar Partida o Empezar de Cero)
if best_pt_path.exists():
    print(f"🎮 'Save Game' detectado. Cargando progreso desde: {best_pt_path}")
    model = YOLO(str(best_pt_path))
    lr_inicial = 0.001 # Tasa más baja para no arruinar lo ya aprendido
else:
    print("🆕 No hay partida guardada. Iniciando entrenamiento desde cero (YOLOv8n).")
    model = YOLO("yolov8n.pt")
    lr_inicial = 0.01

# 3. Entrenamiento
print(f"🚀 Iniciando entrenamiento (Máximo {epochs} epochs)...")
results = model.train(
    data=data_yaml,
    epochs=epochs,
    imgsz=img_size,
    batch=batch_size,
    project=output_dir,
    name="train_experiment",
    exist_ok=True,
    lr0=lr_inicial,
    patience=30,      # Si deja de mejorar por 30 epochs, se detiene
    save=True,
    pretrained=True
)

# 4. Gestión de archivos y Auto-Descarga ("Guardado Final")
print("✅ Entrenamiento finalizado.")

# Ruta donde YOLO acaba de guardar el mejor modelo de esta sesión
new_best_path = Path(output_dir) / "train_experiment" / "weights" / "best.pt"

if new_best_path.exists():
    # 4a. Sobreescribir el Save Game en /content/in (Para la próxima vez que corras el bloque)
    shutil.copy2(new_best_path, best_pt_path)
    print(f"⭐ 'Save Game' actualizado localmente en: {best_pt_path}")

    # 4b. Descargar el archivo al PC automáticamente
    try:
        print("📥 Iniciando descarga del modelo 'best.pt' a tu ordenador...")
        files.download(str(best_pt_path))
        print("✅ Descarga iniciada. ¡Guarda bien este archivo!")
    except Exception as e:
        print(f"⚠️ No se pudo iniciar la descarga automática: {e}")
        print("ℹ️ Puedes descargarlo manualmente desde la carpeta /content/in en el panel izquierdo.")
else:
    print("⚠️ Error: No se encontró el archivo generado en esta sesión.")

🆕 No hay partida guardada. Iniciando entrenamiento desde cero (YOLOv8n).
🚀 Iniciando entrenamiento (Máximo 300 epochs)...
Ultralytics 8.4.9 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=300, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=train

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Descarga iniciada. ¡Guarda bien este archivo!


##
$$\textbf{Bloque 13 — Validación visual en imágenes (val):} \
\\\ \text{Usa best.pt para predecir sobre images/val y guarda imágenes con cajas dibujadas y las descarga.}$$


In [ ]:
# Bloque 13 — Predicciones sobre val

from google.colab import files

# --- 1. CONFIGURACIÓN DE RUTAS ---
base_path = Path("/content")
dir_dataset = base_path / "dataset"
dir_runs = base_path / "runs"
dir_out = base_path / "out"
dir_in = base_path / "in"

# Ruta del modelo y de las imágenes
best_model_path = dir_in / "best.pt"
val_images_path = dir_dataset / "images/val"

# Carpeta de salida
pred_name = "predict_val"
final_pred_dir = dir_runs / pred_name

# --- 2. VALIDACIÓN ---
if not best_model_path.exists():
    print(f"⚠️ No se encuentra el modelo en {best_model_path}. Revisa el Bloque 12.")
elif not val_images_path.exists():
    print(f"⚠️ No existe la carpeta de validación en {val_images_path}")
else:
    try:
        print(f"🚀 Iniciando predicción visual con líneas delgadas...")
        model = YOLO(str(best_model_path))

        # Ejecutamos la predicción con ajustes visuales
        model.predict(
            source=str(val_images_path),
            save=True,
            project=str(dir_runs),
            name=pred_name,
            exist_ok=True,
            conf=0.25,         # Solo muestra lo que tenga > 25% certeza
            iou=0.3,          # Umbral de solapamiento (ajustar si hay cajas duplicadas)
            # --- AJUSTES PARA QUE NO SE TAPEN LAS FALLAS ---
            line_width=3,
            show_labels=True,  # Muestra el nombre de la falla
            show_conf=False,   # Quitamos el % de confianza para limpiar la imagen
            save_txt=False     # No necesitamos los .txt aquí, solo las fotos
        )

        print(f"✅ Fotos guardadas en: {final_pred_dir}")

        # --- 3. CREACIÓN DE ZIP Y DESCARGA ---
        zip_file_path = dir_out / "val_predictions_clean.zip"
        dir_out.mkdir(parents=True, exist_ok=True)

        # Empaquetamos solo las imágenes resultantes
        with zipfile.ZipFile(zip_file_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
            # Buscamos extensiones comunes de imagen
            for ext in ['*.jpg', '*.jpeg', '*.png']:
                for img_file in final_pred_dir.rglob(ext):
                    zipf.write(img_file, arcname=img_file.name)

        if zip_file_path.stat().st_size > 0:
            print(f"📦 ZIP creado: {zip_file_path}")
            files.download(str(zip_file_path))
            print("📥 Descarga iniciada.")
        else:
            print("⚠️ El ZIP está vacío. ¿Seguro que hubo detecciones?")

    except Exception as e:
        print(f"❌ Error: {e}")

#
$$\textbf{Modelo DAICH}$$

##
$$\textbf{Bloque 15 — Motor de Procesamiento de Video:} \ \text{Analiza el video, sincroniza el tiempo con el archivo SRT y ejecuta YOLO.}$$

In [ ]:
def parse_srt(srt_path):                                          # Función para convertir SRT en tabla de datos
    with open(srt_path, 'r', encoding='utf-8') as f:              # Abre el archivo SRT en modo lectura
        content = f.read()                                        # Lee todo el contenido del archivo
    blocks = content.split('\n\n')                                # Divide el contenido por bloques de tiempo
    mapping = []                                                  # Lista para almacenar el mapeo Segundo-KM
    for b in blocks:                                              # Recorre cada bloque del subtítulo
        lines = b.split('\n')                                     # Divide el bloque en líneas individuales
        if len(lines) >= 3:                                       # Verifica que el bloque tenga tiempo y texto
            time_line = lines[1].split(' --> ')[0]                # Extrae el tiempo de inicio de la falla
            h, m, s = time_line.replace(',', '.').split(':')      # Formatea el tiempo para extraer segundos
            total_sec = int(h)*3600 + int(m)*60 + float(s)        # Calcula el tiempo total en segundos
            km_match = re.search(r"(\d+\.?\d*)", lines[2])        # Busca el valor numérico del kilometraje en el texto
            if km_match:                                          # Si encuentra un número de kilometraje
                mapping.append({'seg': total_sec, 'km': float(km_match.group(1))}) # Guarda el par Segundo-KM
    return pd.DataFrame(mapping)                                  # Devuelve un DataFrame con la telemetría

def ejecutar_deteccion(video_p, model_p, srt_p, conf_val):        # Función principal de análisis visual
    model = YOLO(model_p)                                         # Carga el modelo entrenado (.pt)
    df_srt = parse_srt(srt_p)                                     # Procesa el archivo SRT para obtener el kilometraje
    cap = cv2.VideoCapture(str(video_p))                          # Abre el archivo de video para lectura
    fps = cap.get(cv2.CAP_PROP_FPS)                               # Obtiene los frames por segundo del video
    detecciones = []                                              # Lista para almacenar los resultados encontrados
    f_count = 0                                                   # Contador de frames procesados

    print("🚀 Iniciando escaneo de video...")                     # Aviso de inicio de proceso
    while cap.isOpened():                                         # Mientras el video sea legible
        ret, frame = cap.read()                                   # Lee el siguiente frame del video
        if not ret: break                                         # Sale del bucle si el video termina
        if f_count % int(fps) == 0:                               # Procesa solo un frame por cada segundo de video
            seg = f_count / fps                                   # Calcula el segundo actual del video
            km_act = np.interp(seg, df_srt['seg'], df_srt['km'])   # Estima el KM exacto interpolando los datos del SRT
            res = model.predict(frame, conf=conf_val, verbose=False) # Ejecuta YOLO en el frame actual
            for r in res:                                         # Itera sobre los resultados de la detección
                if len(r.boxes) > 0:                              # Si el modelo encontró alguna falla
                    img_n = f"det_{f_count}.jpg"                  # Define nombre de archivo para la captura
                    cv2.imwrite(str(PATHS["frames"]/img_n), r.plot()) # Guarda la imagen con la caja de detección dibujada
                    for b in r.boxes:                             # Itera sobre cada caja detectada en el frame
                        detecciones.append({'km': km_act, 'clase': model.names[int(b.cls)], 'img': img_n}) # Guarda datos
            print(f"⏳ Procesando KM: {km_act:.3f}...")           # Muestra progreso en tiempo real
        f_count += 1                                              # Incrementa el contador de frames
    cap.release()                                                 # Libera el archivo de video de la memoria
    return pd.DataFrame(detecciones)                              # Devuelve la tabla de detecciones visuales

$$\textbf{Bloque 16 — Fusión Técnica e Interpolación Condicional:}
\
\\
\text{Cruza las detecciones visuales con los registros del robot}$$

In [ ]:
# --- CONFIGURACIÓN TÉCNICA ---
X_METROS = 5.0                                                    # Tu regla: si la distancia entre registros es < X, interpola
csv_losa = PATHS["in"] / "losa.csv"                               # Archivo CSV de fondo
csv_para = PATHS["in"] / "paramento.csv"                          # Archivo CSV de muros

df_l = pd.read_csv(csv_losa)                                      # Carga datos de losa
df_p = pd.read_csv(csv_para)                                      # Carga datos de paramento
datos_consolidados = []                                           # Lista para el reporte final

print(f"🧠 Sincronizando con columna 'metros'. Primer registro: {df_l['metros'].iloc[0]}")

for _, det in df_vis.iterrows():                                  # Recorre cada falla detectada por YOLO
    # Selecciona la tabla correcta (Paramento o Losa)
    db = df_p if "paramento" in det['cls'].lower() else df_l

    # Calcula la diferencia absoluta entre el metro de la falla y los registros del CSV
    db['dist_al_punto'] = (db['metros'] - det['km']).abs()

    # Obtiene los 2 registros del robot más cercanos a ese metro exacto
    puntos = db.nsmallest(2, 'dist_al_punto').sort_values('metros')

    # Calcula la distancia real entre esos dos registros del robot
    distancia_entre_registros = puntos['metros'].diff().iloc[-1]

    # --- APLICACIÓN DE LA REGLA DE INTERPOLACIÓN ---
    if distancia_entre_registros < X_METROS:                      # SI LA DISTANCIA ES MENOR A X:
        # Calcula el valor exacto basado en la pendiente entre los dos puntos
        v_ancho = np.interp(det['km'], puntos['metros'], puntos['Ancho'])
        v_prof = np.interp(det['km'], puntos['metros'], puntos['Profundidad'])
        metodo = "Interpolación"
    else:                                                         # SI LA DISTANCIA ES MAYOR A X:
        # Toma el registro más cercano sin promediar
        punto_cercano = db.nsmallest(1, 'dist_al_punto').iloc[0]
        v_ancho = punto_cercano['Ancho']
        v_prof = punto_cercano['Profundidad']
        metodo = "Punto más cercano"

    # Guarda el resultado técnico consolidado
    datos_consolidados.append({**det, 'a': v_ancho, 'p': v_prof, 'metodo': metodo})

df_final = pd.DataFrame(datos_consolidados)                       # Crea la tabla maestra final
print(f"✅ Fusión completada. Procesados {len(df_final)} hallazgos usando la regla de {X_METROS}m.")

#
$$\textbf{Bloques auxiliares}$$

##
$$\textbf{Bloque 3 — Subida de archivos (cvat zip, videos, best.pt):} \
\\\ \text{Permite subir archivos desde tu computador a Colab y los deja guardados en content in para usarlos en el flujo.}$$

In [6]:
print("📂 Selecciona archivos para añadir (Video, SRT, CSVs o Modelo)")     # Aviso de carga
subidos = files.upload()                                                   # Selector de archivos

for nombre_original in subidos.keys():                                     # Itera sobre los archivos subidos
    nombre_min = nombre_original.lower()                                   # Normaliza a minúsculas para comparar

    # --- LÓGICA DE CLASIFICACIÓN (Mantiene archivos existentes) ---
    if "losa" in nombre_min and nombre_original.endswith('.csv'):          # Identifica CSV de fondo
        ruta_destino = PATHS["in"] / "losa.csv"                            # Renombra para el motor de fusión
        print(f"🎯 Identificado como LOSA: {nombre_original}")

    elif "paramento" in nombre_min and nombre_original.endswith('.csv'):   # Identifica CSV de muros
        ruta_destino = PATHS["in"] / "paramento.csv"                       # Renombra para el motor de fusión
        print(f"🎯 Identificado como PARAMENTO: {nombre_original}")

    elif nombre_original.endswith('.pt'):                                 # Identifica modelos YOLO
        ruta_destino = PATHS["in"] / "best.pt"                             # Estandariza nombre del modelo
        print(f"🧠 Modelo cargado/actualizado: {nombre_original}")

    else:                                                                  # Video, SRT y otros
        ruta_destino = PATHS["in"] / nombre_original                       # Conserva nombre original
        print(f"📦 Archivo guardado: {nombre_original}")

    # --- GUARDADO EN DISCO ---
    with open(ruta_destino, "wb") as f:                                    # Abre destino sin borrar el resto de /in
        f.write(subidos[nombre_original])                                  # Escribe el nuevo contenido

📂 Selecciona archivos para añadir (Video, SRT, CSVs o Modelo)


##
$$\textbf{Bloque 6 — Video → Frames:} \
\\\ \text{Si hay un video, extrae imágenes (frames) a una frecuencia definida (fps) y las guarda en #/content/work/frames para etiquetarlas después.}$$


In [ ]:
# Busca un video por extensión común (si existe)
video = next((p for p in in_files if p.suffix.lower() in [".mp4", ".mov", ".avi", ".mkv"]), None)  # Encuentra el primer video

# Extrae frames desde el video detectado (si no hay video, no falla: solo avisa)
fps_extract = 1.0                                              # Define cuántas imágenes por segundo extraer (ej: 1.0 = 1 frame/seg)
img_ext = "jpg"                                                # Define el formato de imagen de salida (jpg o png)

# Si no hay video, no hacemos nada y seguimos
if video is None:                                              # Revisa si se detectó un video en /content/in
    print("⚠️ No hay video detectado en /content/in, así que no se extraen frames todavía.")  # Aviso sin romper el flujo
else:
    # Limpia frames anteriores para no mezclar corridas
    old_frames = list(FRAMES_DIR.glob(f"*.{img_ext}"))          # Busca frames antiguos en la carpeta de frames
    for f in old_frames:                                        # Recorre frames antiguos
        f.unlink()                                              # Borra cada frame antiguo

    # Define el número inicial desde el cual deseas comenzar la numeración
    start_number = 886  # Puedes poner el número que desees

    # Define el patrón de nombres con el número de inicio
    out_pattern = str(FRAMES_DIR / f"%06d.{img_ext}")

    # Construye y ejecuta el comando ffmpeg para extraer frames
    cmd = [
    "ffmpeg", "-y",
    "-i", str(video),
    "-vf", f"fps={fps_extract}",
    "-start_number", str(start_number), # <--- ESTO indica dónde empezar
    out_pattern]

    print("✅ Ejecutando FFmpeg para extraer frames:")          # Mensaje de inicio
    print("   ", " ".join(cmd))                                 # Muestra el comando para trazabilidad
    subprocess.run(cmd, check=False)                            # Ejecuta sin romper el notebook si ffmpeg devuelve error

    # Cuenta y muestra resultados
    frames = sorted(FRAMES_DIR.glob(f"*.{img_ext}"))            # Busca los frames generados
    if len(frames) == 0:                                        # Si no se generó ningún frame
        print("⚠️ No se generaron frames (revisa si el video está OK o si fps_extract es muy bajo).")  # Aviso
    else:
        print(f"✅ Frames listos: {len(frames)} en {FRAMES_DIR}")  # Confirma cuántos frames se crearon
        print("   Ejemplo primero/último:", frames[0].name, "|", frames[-1].name)  # Muestra nombres de ejemplo

# Comprime los frames en un ZIP descargable (si no hay frames, no falla: solo avisa)
zip_frames_path = DIR_OUT/"frames.zip"                         # Define dónde quedará el zip final

# Busca frames existentes en la carpeta de frames
frame_files = sorted(list(FRAMES_DIR.glob("*.jpg")) + list(FRAMES_DIR.glob("*.png")))  # Reúne frames jpg/png

# Si no hay frames aún, avisa y termina sin error
if len(frame_files) == 0:                                      # Revisa si hay frames para comprimir
    print("⚠️ No hay frames en /content/work/frames, así que no se puede crear frames.zip todavía.")  # Aviso
else:
    # Si ya existía un zip antiguo, lo borra para evitar confusiones
    if zip_frames_path.exists():                               # Verifica si el zip ya existe
        zip_frames_path.unlink()                               # Borra el zip anterior

    # Crea el zip con todos los frames
    with zipfile.ZipFile(zip_frames_path, "w", zipfile.ZIP_DEFLATED) as z:  # Abre un zip en modo escritura
        for f in frame_files:                                  # Recorre cada frame
            z.write(f, arcname=f.name)                         # Agrega el archivo al zip con su nombre

    # Confirma que el zip se creó correctamente
    print("✅ ZIP creado para CVAT:", zip_frames_path)          # Muestra la ruta del zip creado
    print(f"✅ Incluye {len(frame_files)} imágenes.")          # Indica cuántas imágenes quedaron dentro

    # Opción de descarga directa (si quieres)
    try:
        from google.colab import files                         # Importa herramienta de descarga de Colab
        files.download(str(zip_frames_path))                   # Descarga el zip a tu PC
        print("✅ Descarga iniciada (si tu navegador lo permite).")  # Confirmación
    except Exception:
        print("⚠️ No pude iniciar descarga automática, pero el zip quedó en /content/out para descargarlo manualmente.")  # Aviso



✅ Ejecutando FFmpeg para extraer frames:
    ffmpeg -y -i /content/in/TRAMO 3 - PARTE 2.mp4 -vf fps=1.0 -start_number 590 /content/work/frames/%06d.jpg
✅ Frames listos: 296 en /content/work/frames
   Ejemplo primero/último: 000590.jpg | 000885.jpg
✅ ZIP creado para CVAT: /content/out/frames.zip
✅ Incluye 296 imágenes.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Descarga iniciada (si tu navegador lo permite).
